In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

import pandas as pd
import numpy as np

from typing import Optional

import os

from dataloader import AllBeautyLoader

<b>check_features</b> - автоматизация интересующего нас EDA выборок из Amazon: AllBeauty.
<ul>
   Мы фиксируем желаемый размер выборки и семя случайной генерации и смотрим на:
    <li>Распределение оценок товаров в выборке</li>
    <li>Распределение факта покупки товара в выборке</li>
</ul>
Устраивает нас распределение или нет мы решаем при помощи искуственно заданного распределения(по умлчанию - равномерное) и по допустимому отклонению двух фактических статистик от этого распределения(eps_1 и eps_2). Результат анализа сохраняем в excel.

In [2]:
# Идея - автоматизировать просмотр выборок
# uniform +- epsilon
def check_features(
        input_path: str, seeds: list[int],
        sizes: list[int], save: Optional[str]=None,
        dist: str='uniform', eps_1: float=0.05, eps_2: float=0.05):

    if not os.path.exists(input_path):
        raise ValueError('input_path не существует!')
    
    hist = [] if save else None
    
    # возвращает требуемый балланс(распределение) классов в выборке
    # [('rating'), ('verified_purchase')]
    distributions = {
        'uniform': [np.array([0.2, 0.2, 0.2, 0.2, 0.2]), np.array([0.5, 0.5])]
    }

    d = distributions[dist]
    
    for seed in seeds:
        for size in sizes:
            res = 0
            loader = AllBeautyLoader(input_path, sample_size=size, seed=seed)
            sample = loader.get_sample()
            all_stats = pd.DataFrame(sample[['rating', 'verified_purchase']].value_counts())
            r_stats = sample['rating'].value_counts().sort_index().values / size
            p_stats = sample['verified_purchase'].value_counts().sort_index().values / size
            try:
                if (np.abs(d[0] - r_stats) < eps_1).all() or (np.abs(d[1] - p_stats) < eps_2).all():
                    print('Dist: ', f'rating - {r_stats}, ', f'verified_purchase - {p_stats}')
                    print('Seed: ', seed)
                    print('Size: ', size)
                    res = 1
            except:
                print('Плохая выборка!')
                print('Dist: ', f'rating - {r_stats}, ', f'verified_purchase - {p_stats}')
                print('Seed: ', seed)
                print('Size: ', size)
            if hist is not None:
                raw = [r_stats, p_stats, seed, size, res]
                rating_raw = pd.DataFrame(raw[0].reshape(-1, 5), columns=['Rating 1', 'Rating 2', 'Rating 3', 'Rating 4', 'Rating 5'])
                purchase_raw = pd.DataFrame(raw[1].reshape(-1, 2), columns=['Verified purchase 0', 'Verified purchase 1'])
                seed_raw = pd.DataFrame([raw[2]], columns=['Seed'])
                size_raw = pd.DataFrame([raw[3]], columns=['Size'])
                res_raw = pd.DataFrame([raw[4]], columns=['Criterion result'])
                hist.append(pd.concat([
                    rating_raw, purchase_raw, seed_raw, size_raw, res_raw
                ], axis=1))
                
    if hist is not None:
        hist_excel = hist[0]
        for h in hist[1:]:
            hist_excel = pd.concat([hist_excel, h], axis=0)
        hist_excel.to_excel(save, index=False)
    return

In [3]:
# задаем исследуемые сиды и размеры выборок
seeds = [i for i in range(1)]
sizes = [100, 200]

In [4]:
# задаем распределения и допустимые отклонения
dist = 'uniform'
eps_1, eps_2 = 0.1, 0.1

In [5]:
# настройка путей
input_path = '/home/roman/Документы/AmazonRecomendationSystem/data/raw/all_beauty/All_Beauty.jsonl'
output_path = os.path.join(os.getcwd(), f'sample_stats(dist_{dist}_eps_1_{eps_1}_eps_2_{eps_2}).xlsx')

In [6]:
# запускаем сбор статистики
check_features(
    input_path=input_path, seeds=seeds,
    sizes=sizes, save=output_path,
    dist=dist, eps_1=eps_1, eps_2=eps_2
)

Загрузка данных: 100%|██████████████████████| 200/200 [00:00<00:00, 9499.69it/s]
